# Experimental Evaluation: 
## Goal Recognition in FOND using Goals Formalized in LTLf (Until)

In [177]:
BASE_DIR = '../fond-recognition-benchmarks'

In [178]:
import os
import sys
import pandas as pd
import glob
import json
import numpy as np
import itertools

In [179]:
def compute_posterior(index, prob, priors):
    num = prob[index] * priors[index]
    denom = np.sum(prob*priors)
    return num / denom

In [180]:
RESULTS_DIR = os.path.join(BASE_DIR, 'results/')

In [181]:
domains_evaluation = ['blocksworld', 'logistics', 'tidyup', 'tireworld', 'triangle-tireworld', 'zenotravel']
type_goal = 'ltl_until'
json_files = []
for domain in domains_evaluation:
    files = glob.glob(os.path.join(RESULTS_DIR + '/' + domain + '/' + type_goal, '*.json'))
    for f in files:
        json_files.append(f)

In [182]:
print("Results found:", len(json_files))

Results found: 468


# Results Collection

In [183]:
print(json_files[0])
print(os.path.basename(json_files[0]))
filename, ext = os.path.basename(json_files[0]).split('.')

../fond-recognition-benchmarks/results//blocksworld/ltl_until/blocksworld_ltl2_p01_hyp-2_50_2.json
blocksworld_ltl2_p01_hyp-2_50_2.json


In [184]:
tokens = filename.split('_')

In [185]:
dataset = {
    'domain' : [],\
    'problem' : [],\
    'observability' : [],\
    'goals' : [],\
    'judge_point' : [],\
    'obs_len' : [],\
    'true_goal': [],\
    'time': [],\
    'posterior': []}

sum_obs_dict = dict()
problems_obs_dict = dict()

sum_goals = 0

for pathname in json_files:
    filename, ext = os.path.basename(pathname).split('.')
    tokens = filename.split('_')
    approach = tokens[0]
    model_type = tokens[-1]
    data = {}
    with open(pathname) as instream:
        buffer = instream.read()
        data = json.loads(buffer)
    #print(approach, model_type, data['domain'], data['problem'], data['observability'])
    num_goals = len(data["G"])
    true_goal = data["G"].index(data['G*'])
    likelihoods = data['P(Obs | G)']
    sum_goals += num_goals
    
    if data['observability'] in sum_obs_dict:
        problems_obs_dict[data['observability']] += 1
        sum_obs_dict[data['observability']] += len(data['Obs'])
    else:
        problems_obs_dict[data['observability']] = 1
        sum_obs_dict[data['observability']] = len(data['Obs'])
    
    for k, prob_O_G in enumerate(likelihoods):
        post_probs = [compute_posterior(j, prob_O_G, np.ones(num_goals)/num_goals) \
                     for j in range(len(data['G']))]
        dataset['domain'] += [data['domain']]
        dataset['problem'] += [data['problem']]
        dataset['goals'] += [num_goals]
        dataset['judge_point'] += [k]
        dataset['obs_len'] += [len(data['Obs'])]
        dataset['true_goal'] += [true_goal]
        dataset['observability'] += [data['observability']]
        dataset['time'] += [data['time']]
        dataset['posterior'] += [np.array(post_probs)]

avg_obs = 0.0
sum_obs = 0.0
avg_obs_observability = dict()
for k in sum_obs_dict.keys():
    avg = sum_obs_dict[k] / problems_obs_dict[k]
    avg_obs_observability[k] = avg
    sum_obs += avg
    
avg_obs = sum_obs / len(sum_obs_dict)

avg_goals = sum_goals/len(json_files)

In [186]:
dataset = pd.DataFrame(dataset)

In [187]:
def compute_accuracy(dataset, top_k):
    tp_count = 0
    for index, row in dataset.iterrows():
        top_k_goals = np.argpartition(row['posterior'], -top_k)[-top_k:]
        if row['true_goal'] in top_k_goals:
            tp_count += 1
    return tp_count / len(dataset)

In [188]:
def compute_recognition_time(dataset):
    sum_time = 0
    for index, row in dataset.iterrows():
        sum_time += row['time']

    return (sum_time / len(dataset))

In [189]:
def compute_error_rates(dataset):
    tpr = 0.0
    fpr = 0.0
    fnr = 0.0

    for index, row in dataset.iterrows():
        tp_count = 0
        tn_count = 0
        fp_count = 0
        fn_count = 0

        top_goal = np.argmax(row['posterior'])
        max_goals = [ k for k, p in enumerate(row['posterior']) if p == row['posterior'][top_goal]]
        
        if row['true_goal'] in max_goals:
            tp_count = 1
        fn_count = 1 - tp_count
        fp_count = len(max_goals) - tp_count
        tn_count = row['goals'] - fp_count
    
        tpr += tp_count/(tp_count + fn_count)
        fpr += fp_count/(fp_count + tn_count)
        fnr += fn_count/(fn_count + tp_count)
    return tpr/len(dataset), fpr/len(dataset), fnr/len(dataset)

## Online Goal Recognition

In [190]:
approaches = ['approach']
observabilities = ['10', '25', '30', '50', '70', '75', '100']

summary = {'approach': [], 'observability': [], 'top1': [], 'top2': [], 'top3': [], 'tpr' : [], 'fpr':[], 'fnr': []}

for approach, observability in itertools.product(approaches, observabilities):
    df = dataset.loc[(dataset['observability'] == observability)]
    if len(df) == 0: 
        continue
    top_1_acc = compute_accuracy(df, 1)
    top_2_acc = compute_accuracy(df, 2)
    top_3_acc = compute_accuracy(df, 3)
    tpr, fpr, fnr = compute_error_rates(df)
    summary['approach'] += [approach]
    summary['observability'] += [observability]
    summary['top1'] += [top_1_acc]
    summary['top2'] += [top_2_acc]
    summary['top3'] += [top_3_acc]
    summary['tpr'] += [tpr]
    summary['fpr'] += [fpr]
    summary['fnr'] += [fnr]

In [191]:
summary = pd.DataFrame(summary)

In [192]:
summary[['observability', 'top1', 'top2', 'top3', 'tpr', 'fpr', 'fnr']]

,observability,top1,top2,top3,tpr,fpr,fnr
0,10,0.548718,0.830769,0.948718,0.738462,0.153419,0.261538
1,30,0.547059,0.823529,0.900000,0.721569,0.180556,0.278431
2,50,0.543071,0.818976,0.941323,0.720350,0.179359,0.279650
3,70,0.523307,0.798593,0.925242,0.712401,0.184110,0.287599
4,100,0.525097,0.797297,0.916988,0.710425,0.186937,0.289575


## Offline Goal Recognition

In [193]:
offline = {'observability': [], 'avg_obs': [], 'time': [], 'top1': [], 'top2': [], 'top3': [], 'tpr' : [], 'fpr':[], 'fnr': []}

for approach, observability in itertools.product(approaches, observabilities):
    df = dataset.loc[(dataset['observability'] == observability)\
                    & (dataset['judge_point'] == dataset['obs_len'] - 1)]
    if len(df) == 0: continue
    top_1_acc = compute_accuracy(df, 1)
    top_2_acc = compute_accuracy(df, 2)
    top_3_acc = compute_accuracy(df, 3)
    tpr, fpr, fnr = compute_error_rates(df)
    avg_time = compute_recognition_time(df)
    offline['observability'] += [observability]
    offline['avg_obs'] += [avg_obs_observability[observability]]
    offline['time'] += [avg_time]
    offline['top1'] += [top_1_acc]
    offline['top2'] += [top_2_acc]
    offline['top3'] += [top_3_acc]
    offline['tpr'] += [tpr]
    offline['fpr'] += [fpr]
    offline['fnr'] += [fnr]

In [194]:
offline = pd.DataFrame(offline)
offline[['observability', 'avg_obs', 'time', 'top1', 'top2', 'top3', 'tpr', 'fpr', 'fnr']]

,observability,avg_obs,time,top1,top2,top3,tpr,fpr,fnr
0,10,1.805556,147.983845,0.648148,0.944444,0.990741,0.796296,0.108025,0.203704
1,30,4.722222,146.487498,0.740741,0.944444,0.972222,0.916667,0.113426,0.083333
2,50,7.416667,142.654668,0.768519,0.962963,0.990741,0.935185,0.104167,0.064815
3,70,10.527778,143.651607,0.787037,0.944444,1.000000,0.972222,0.099537,0.027778
4,100,14.388889,141.382081,0.805556,1.000000,1.000000,0.972222,0.076389,0.027778


In [195]:
print('Average observations: %s' % avg_obs)
print('Average goal: %s' % avg_goals)


Average observations: 7.772222222222223
Average goal: 3.9166666666666665


In [196]:
columns = ['observability', 'avg_obs', 'time', 'tpr', 'fpr', 'fnr']
summary4tex = offline[columns]
with open('summary-ltl_until.tex','w') as output :
    summary4tex.to_latex(buf=output,index=False,float_format=lambda x : '{:.2f}'.format(x))

### First Observation

In [197]:
first_obs = {'observability': [], 'top1': [], 'top2': [], 'top3': [], 'tpr' : [], 'fpr':[], 'fnr': []}

for approach, observability in itertools.product(approaches, observabilities):
    df = dataset.loc[(dataset['observability'] == observability)\
                    & (dataset['judge_point'] == 0)]
    if len(df) == 0: continue
    #print(len(df))
    top_1_acc = compute_accuracy(df, 1)
    top_2_acc = compute_accuracy(df, 2)
    top_3_acc = compute_accuracy(df, 3)
    tpr, fpr, fnr = compute_error_rates(df)
    first_obs['observability'] += [observability]
    first_obs['top1'] += [top_1_acc]
    first_obs['top2'] += [top_2_acc]
    first_obs['top3'] += [top_3_acc]
    first_obs['tpr'] += [tpr]
    first_obs['fpr'] += [fpr]
    first_obs['fnr'] += [fnr]

In [198]:
first_obs = pd.DataFrame(first_obs)
first_obs[['observability', 'top1', 'top2', 'top3', 'tpr', 'fpr', 'fnr']]

,observability,top1,top2,top3,tpr,fpr,fnr
0,10,0.546296,0.805556,0.944444,0.703704,0.149691,0.296296
1,30,0.481481,0.814815,0.879630,0.592593,0.164352,0.407407
2,50,0.453704,0.805556,0.972222,0.574074,0.166667,0.425926
3,70,0.435185,0.759259,0.935185,0.555556,0.164352,0.444444
4,100,0.444444,0.777778,0.944444,0.555556,0.155093,0.444444
